## Heuristic Benchmark

This notebook demonstrates the Heuristic Benchmark design pattern


### 1. Regression on poorly understood features

Problem: Time interval before a question on Stack Overflow is answered.

Benchmark: Median time to first answer over the entire training dataset, so 2120 seconds.

In [1]:
%%bigquery
SELECT 
  bqutil.fn.median(ARRAY_AGG(TIMESTAMP_DIFF(a.creation_date, q.creation_date, SECOND))) AS time_to_answer
FROM `bigquery-public-data.stackoverflow.posts_questions` q
JOIN `bigquery-public-data.stackoverflow.posts_answers` a
ON q.accepted_answer_id = a.id

,time_to_answer
0,2120.0


Find the error metric of always predicting that it will take 2120 seconds to get an answer. This the baseline metric against which to report model performance.

In [2]:
%%bigquery
WITH benchmark_eval AS (
SELECT 
  2120 - TIMESTAMP_DIFF(a.creation_date, q.creation_date, SECOND) AS error
FROM `bigquery-public-data.stackoverflow.posts_questions` q
JOIN `bigquery-public-data.stackoverflow.posts_answers` a
ON q.accepted_answer_id = a.id
)

SELECT
   AVG(ABS(error)) AS mean_absolute_error
FROM
   benchmark_eval

,mean_absolute_error
0,857315.119106


### 2. Classification on poorly understood features


Problem: Whether or not an accepted answer will be edited.

Benchmark: Probability distribution of accepted answers that are edited.

In [3]:
%%bigquery
SELECT 
  AVG(IF(a.last_edit_date IS NULL, 0, 1)) AS prob_edited
FROM `bigquery-public-data.stackoverflow.posts_questions` q
JOIN `bigquery-public-data.stackoverflow.posts_answers` a
ON q.accepted_answer_id = a.id

,prob_edited
0,0.36226


Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License